# Семинар 20
# Методы внутренней точки

## На прошлом семинаре

- Метод проекции градиента
- Метод условного градиента

## Задача выпуклой оптимизации для ограничений типа равенств


## Двойственная задача

## Метод Ньютона для задач с ограничениями типа равенств

### ... или линеаризация условий оптимальности

### Теорема сходимости

## Случай недопустимой начальной точки

### Прямо-двойственная интерпретация

### Способ инициализации

### Теорема сходимости

## Примеры

## Общая задача выпуклой оптимизации

## Условия оптимальности

## Логарифмический барьер

## Центральный путь

## Физическая интерпретация

## Барьерный метод

### Выбор параметров

### Теорема сходимости

## Задача поиска допустимого начального приближения

### Первая фаза метода

### Вторая фаза метода

### Комбинирование первой фазы и второй фазы

### Общая сложность метода

## Прямо-двойственный метод

### Сравнение с барьерным методом

## Резюме

- Метод Ньютона для выпуклой задачи с оганичениями типа равенств
- Случай недопустимой начальной точки
- Прямой барьерный метод
- Прямо-двойственный метод